In [ ]:
import pandas as pd
training_df = pd.read_csv('./input_data/final_training_set_v2.csv')
validation_df = pd.read_csv('./input_data/final_validation_set_v2.csv')

In [ ]:
training_df

,text,labels
0,சுற்றத்தை கெடுத்து gay ahh வாழ்வதை விட அடுத்த...,Homophobia
1,ટ્રાન્સજેન્ડરો સમાજના મૂલ્યોનો અનાદર કરે છે.,Transphobia
2,તમે ટ્રાન્સજેન્ડર છો? તમને શાને કરવામાં આવે છે?,Transphobia
3,పెళ్లిపై ఆమిర్‌కు సల్మాన్ గట్టి కౌంటర్!,Non-anti-LGBT+ content
4,તેમની જાતિ કેવી રીતે તેમની જેન્ડર ઓળખને પ્રભાવ...,Transphobia
...,...,...
75031,Rajeswari Rajagopalan Sister Last week when I ...,Transphobia
75032,તમે ટ્રાન્સજેન્ડર છો એટલે તમને અસલી માણસ તરીકે...,Transphobia
75033,Venkat Raman போடா மெண்டல் 9 துங்ககிட்ட நீ அசி...,Transphobia
75034,இது வரவேற்கத்தக்க தீர்ப்பு ஏனென்றால் தன்பாலின ...,Non-anti-LGBT+ content


In [ ]:
def pls(x):
    if x == 'Homophobia':
        return 0
    if x == 'Transphobia':
        return 1
    return 2

In [ ]:
training_df['labels_nb'] = training_df['labels'].apply(lambda x: pls(x))

In [ ]:
validation_df['labels_nb'] = validation_df['labels'].apply(lambda x: pls(x))

In [ ]:
corpus_training = list(training_df['text'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(corpus_training)
# X_train_counts.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
# X_train_tfidf.shape

In [ ]:
corpus_training_target = list(training_df['labels_nb'])

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, corpus_training_target)

In [ ]:
corpus_validation = list(validation_df['text'])
corpus_validation_target = list(validation_df['labels_nb'])
X_new_counts = count_vect.transform(corpus_validation)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [ ]:
validation_df['predicted_classes'] = clf.predict(X_new_tfidf)

In [ ]:
corect_prediction = sum(validation_df['labels_nb'] == validation_df['predicted_classes'])

In [ ]:
corect_prediction / len(validation_df)

0.776796467282216

In [ ]:
from sklearn.metrics import f1_score
f1_score(validation_df['labels_nb'], validation_df['predicted_classes'], average='macro')

0.7648854398601016

In [ ]:
from os import listdir
from os.path import isfile, join
path_to_test = './input_data/test'
onlyfiles = [f for f in listdir(path_to_test) if isfile(join(path_to_test, f))]

In [ ]:
onlyfiles = [f'./input_data/test/{file_name}' for file_name in onlyfiles]

In [ ]:
for file in onlyfiles:
    test_df = pd.read_csv(file)
    test_df = test_df.rename(columns=lambda x: x.strip())
    print(file)
    test_df['labels_nb'] = test_df['labels'].apply(lambda x: pls(x))
    corpus_test = list(test_df['text'])
    corpus_test_target = list(test_df['labels_nb'])
    X_new_counts = count_vect.transform(corpus_test)
    X_new_tfidf = tfidf_transformer.transform(X_new_counts)
    predicted_classes = clf.predict(X_new_tfidf)
    f1_result = f1_score(test_df['labels_nb'], predicted_classes, average='macro')

    print(f'For {file}, f1: {f1_result}')

./input_data/test/homo_Kannada_test_1.csv
For ./input_data/test/homo_Kannada_test_1.csv, f1: 0.8671771286862912
./input_data/test/homo_Gujarati_test-2.csv
For ./input_data/test/homo_Gujarati_test-2.csv, f1: 0.8698787062724117
./input_data/test/homo_hin_3_test-1.csv
For ./input_data/test/homo_hin_3_test-1.csv, f1: 0.30508474576271183
./input_data/test/homo_Telugu_test-1.csv
For ./input_data/test/homo_Telugu_test-1.csv, f1: 0.8156887952482061
./input_data/test/Marathi_test-1.csv
For ./input_data/test/Marathi_test-1.csv, f1: 0.4052133172420853
./input_data/test/Homo_Spanish_test-1.csv
For ./input_data/test/Homo_Spanish_test-1.csv, f1: 0.53542232955652
./input_data/test/homo_tam_3_test-1.csv
For ./input_data/test/homo_tam_3_test-1.csv, f1: 0.5709191412727319
./input_data/test/homo_mal_3_test_1.csv
For ./input_data/test/homo_mal_3_test_1.csv, f1: 0.5525332317236121
./input_data/test/homo_eng_3_test-1.csv
For ./input_data/test/homo_eng_3_test-1.csv, f1: 0.4553867595818815
./input_data/test/T

In [ ]:
from google.colab import drive
drive.mount('/content/drive')